# Perform a fit of the data with the Verhulst Model and store the outcome

### Just the run the notebook and then switch to COVID_Italy_Plot to look at the data and fit

In [2]:
# Import the needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import datetime
import csv
from collections import Counter

# Data source: Protezione civile, official Italian goverment press release

### http://www.protezionecivile.gov.it/web/guest/media-comunicazione/comunicati-stampa

### CSV File available on GitHub
https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.

### Infographics from Italian Government
http://opendatadpc.maps.arcgis.com/apps/opsdashboard/index.html#/b0c68bce2cce478eaac82fe38d4138b1


In [3]:
#Load data and define some needed parameters, as the total population
N_ita = 60.48*np.power(10,6)

#Load the regional data into a Pandas DataFrame.
url = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'
covid19_ita_full = pd.read_csv(url)

#Load the National data into another Pandas DataFrame
url_national = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'
covid19_ita_nazionale = pd.read_csv(url_national)

#We are loading the national data as if they were part of a region called "Italia" with code 0
#to have the ability to easily plot the national data

In [4]:
# Filter and adjust the pandas dataframe so that the national data conform with the regional database

#New columns 
covid19_ita_nazionale['codice_regione'] = [0]*covid19_ita_nazionale.shape[0]
covid19_ita_nazionale['denominazione_regione'] = ['Italia']*covid19_ita_nazionale.shape[0]
covid19_ita_nazionale['lat'] = [0]*covid19_ita_nazionale.shape[0]
covid19_ita_nazionale['long'] = [0]*covid19_ita_nazionale.shape[0]
# Reorder the columns
covid19_ita_nazionale = covid19_ita_nazionale[list(covid19_ita_full.columns)]

#Append the national data to the regional database
covid19_ita = covid19_ita_full.append(covid19_ita_nazionale, ignore_index=True)

#Convert the string into a datetime object to keep track of time
covid19_ita['data']=pd.to_datetime(covid19_ita['data'],format="%Y-%m-%d %H:%M:%S") 
#covid19_ita.dtypes

#Retain only interesting data and rename columns
covid19_ita=covid19_ita.drop(columns=['stato', 'lat','long'])

#Translate header in English
covid19_ita.columns = ['date', 'code_region', 'region', 'checked_in','intensive_care','total_admitted','home_isolation','currently_positive','new_positives','recovered','deceased','total_cases','swabs']

# Data Analysis

In [5]:
#Define the function to use for curve_fit. Here we are using the Verhulst model
def Verhulst(t, K, I0, r):
    return (K*I0*np.exp(r*t))/(K+I0*(np.exp(r*t)-1))

In [6]:
# Function to create the numpy array of times associated to the data. The time-scale is in days
def time_vector(data_frame):
    time_frame = data_frame[data_frame.region=='Italia']['date'].to_numpy()
    t_vec = np.zeros(len(time_frame))
    for k in range(len(time_frame)):
        t_vec[k] = ((time_frame[k]-time_frame[0]).item())/(np.power(10,9)*60*60*24)
    return t_vec

In [7]:
# Function to fit the data
def data_analysis_single(region,indicator_to_plot):
    time_Data = time_vector(covid19_ita)
    y_Data = covid19_ita[covid19_ita.region==region][indicator_to_plot].to_numpy()
    fittedParameters, covariance = curve_fit(Verhulst,time_Data,y_Data, bounds=([0,0,0],[600000,1000,1]))
    return fittedParameters, covariance

In [8]:
#Full list of regions and indicators to analyze
region_list = covid19_ita[0:21].region.to_list()
region_list.append('Italia')
indicators = list(covid19_ita.columns.values)[3:11]

## Fit with a Verhulst model and fill a csv with the value of the parameters and their error

### Fixed region

In [9]:
# Fixed Region Fit
for k in region_list:
    with open('./Fit_Data/Regions/Verhulst_Parameters_{}.csv'.format(k), 'w', newline='') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['indicator','K','delta_K','r','delta_r','I0','delta_I0'])
        for n in indicators:
            p_opt, p_cov = data_analysis_single(k,n)
            p_err = np.sqrt(p_cov.diagonal())
            thewriter.writerow([n,p_opt[0],p_err[0],p_opt[2],p_err[2],p_opt[1],p_err[1]])

### Fixed Indicator

In [10]:
# Fixed Indicator Fit
for n in indicators:
    with open('./Fit_Data/Indicators/Verhulst_Parameters_{}.csv'.format(n), 'w', newline='') as f:
        thewriter = csv.writer(f)
        thewriter.writerow(['region','K','delta_K','r','delta_r','I0','delta_I0'])
        for k in region_list:
            p_opt, p_cov = data_analysis_single(k,n)
            p_err = np.sqrt(p_cov.diagonal())
            thewriter.writerow([k,p_opt[0],p_err[0],p_opt[2],p_err[2],p_opt[1],p_err[1]])